# [Module 3.1] Personalize 솔류션 및 솔류션 버전 생성

이 노트북은 Module1에서 생성한 데이타셋 그룹, 데이타셋을 바탕으로 아래와 같은 작업을 합니다.

* 레서피(알고리즘) 선택 및 7가지 솔류션 생성
* 7가지 솔류션 버전 생성

--- 
이 노트북은 약 50분 소요 됩니다.



In [50]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [51]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [52]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [53]:
# suffix = str(np.random.uniform())[4:9]

## 솔루션 및 버전 생성

Amazon Personalize에서 훈련된 모델을 솔루션이라고 하며, 각 솔루션에는 모델이 훈련되었을 때 주어진 데이터량과 관련된 많은 특정 버전들이 있을 수 있습니다.

우선, Amazon Personalize에서 지원되는 모든 레시피(레시피는 아직 데이터에 대해 훈련되지 않은 알고리즘입니다.)들을 나열합니다. 리스트업된 레시피들 중 하나를 선택하고 이를 사용하여 모델을 빌드해 보세요.

이 LAB에서 아래와 같은 모든 솔루션을 생성하여 성능을 비교하여 봅니다.

    1) User-Personalization 
    2) HRNN
    3) HRNN Coldstart
    4) HRNN-Metadata
    5) popularity-count 
    6) SIMS
    7) Personalized Ranking

이 프로세스의 완료는 실제로 40분 이상 소요됩니다. 작업이 완료될 때까지(즉, 활성화 상태가 될 때까지) while 루프를 수행하는 방법도 있지만, 이렇게 하면 다른 셀의 실행을 차단하게 됩니다. 따라서, 많은 모델을 만들어 신속하게 배포하려면 while 루프를 사용하는 대신, 필요한 솔루션 버전들을 생성 후, SageMaker 및 Cloudwatch에서 업데이트를 확인하세요.

###  레시피 리스트 확인

In [54]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


###  User-Personalization 솔루션 생성 

In [55]:
# Recipe 선택 
system_user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-user-personalization-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_user_personalization_recipe_arn,
)

In [56]:
user_personalization_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-60498",
  "ResponseMetadata": {
    "RequestId": "7e7e18e6-e31a-40aa-9328-ac8387826538",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:24 GMT",
      "x-amzn-requestid": "7e7e18e6-e31a-40aa-9328-ac8387826538",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [57]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn
)

user_personalization_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-60498/2ea58de0",
  "ResponseMetadata": {
    "RequestId": "6f6f2799-5ee3-4371-8fae-8a815a1710c2",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:25 GMT",
      "x-amzn-requestid": "6f6f2799-5ee3-4371-8fae-8a815a1710c2",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


###  Popularity Count 솔루션 생성 

In [58]:
# Recipe 선택 
system_popularity_recipe_arn = "arn:aws:personalize:::recipe/aws-popularity-count"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-popularity-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_popularity_recipe_arn,
)



In [59]:
popularity_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-60498",
  "ResponseMetadata": {
    "RequestId": "e60fd9c4-f841-4061-9cc9-f9e3b5b2e36f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:25 GMT",
      "x-amzn-requestid": "e60fd9c4-f841-4061-9cc9-f9e3b5b2e36f",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [60]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = popularity_solution_arn
)

popularity_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-60498/a7709e59",
  "ResponseMetadata": {
    "RequestId": "e5870cc1-a676-41b6-95c6-6183f551e58d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:24 GMT",
      "x-amzn-requestid": "e5870cc1-a676-41b6-95c6-6183f551e58d",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN 솔루션 생성

In [61]:
 # Recipe 선택 
system_hrnn_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"


# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-aws-hrnn-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_hrnn_recipe_arn,
)

hrnn_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-60498",
  "ResponseMetadata": {
    "RequestId": "ae131359-7c9c-44cb-961f-a0e4fb5baac1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:25 GMT",
      "x-amzn-requestid": "ae131359-7c9c-44cb-961f-a0e4fb5baac1",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [62]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

hrnn_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-60498/b1d26512",
  "ResponseMetadata": {
    "RequestId": "7626cbe4-2577-4e1c-987b-0d198d630ff0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:25 GMT",
      "x-amzn-requestid": "7626cbe4-2577-4e1c-987b-0d198d630ff0",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN-Meta 솔루션 생성

In [63]:
 # Recipe 선택 
system_hrnn_meta_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-metadata"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-aws-hrnn-metadata-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_hrnn_meta_recipe_arn,
)

hrnn_meta_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-metadata-60498",
  "ResponseMetadata": {
    "RequestId": "9c52189c-ce5a-4364-b06b-1ed9161e5c2e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:25 GMT",
      "x-amzn-requestid": "9c52189c-ce5a-4364-b06b-1ed9161e5c2e",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [64]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_meta_solution_arn
)

hrnn_meta_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-metadata-60498/765fd88d",
  "ResponseMetadata": {
    "RequestId": "715ec704-8433-48b0-93e4-ac2c3b78cac9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:26 GMT",
      "x-amzn-requestid": "715ec704-8433-48b0-93e4-ac2c3b78cac9",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN ColdStart 솔루션 생성

In [65]:
 # Recipe 선택 
system_hrnn_coldstart_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-coldstart"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-hrnn-coldstart-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_hrnn_coldstart_recipe_arn,
    solutionConfig = {
        "featureTransformationParameters" : {
            'cold_start_max_duration' : '5',
            'cold_start_relative_from' : 'latestItem',
            'cold_start_max_interactions':'15'
        }
    }
)

hrnn_coldstart_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-hrnn-coldstart-60498",
  "ResponseMetadata": {
    "RequestId": "419a6504-7778-476e-a130-8bc520271b1e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:26 GMT",
      "x-amzn-requestid": "419a6504-7778-476e-a130-8bc520271b1e",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [66]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_coldstart_solution_arn
)

hrnn_coldstart_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-hrnn-coldstart-60498/7e3c69db",
  "ResponseMetadata": {
    "RequestId": "21a1ff7b-2aa1-4e80-a30e-d406b07322b7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:26 GMT",
      "x-amzn-requestid": "21a1ff7b-2aa1-4e80-a30e-d406b07322b7",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS 솔루션 생성 

In [67]:
 # Recipe 선택 
system_sims_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-sims-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_sims_recipe_arn
)

sims_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-sims-60498",
  "ResponseMetadata": {
    "RequestId": "dab76e57-3537-464e-b597-2e333b4d6e0e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:26 GMT",
      "x-amzn-requestid": "dab76e57-3537-464e-b597-2e333b4d6e0e",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [68]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)
sims_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-sims-60498/3d08ae29",
  "ResponseMetadata": {
    "RequestId": "80734eee-bf56-4354-9600-43f969c5e208",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:26 GMT",
      "x-amzn-requestid": "80734eee-bf56-4354-9600-43f969c5e208",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalize Ranking 솔루션 생성 

In [69]:
 # Recipe 선택 
system_ranking_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-ranking-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_ranking_recipe_arn
)

ranking_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-60498",
  "ResponseMetadata": {
    "RequestId": "04191d3b-0efc-4a75-95d4-99616281d3f8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:26 GMT",
      "x-amzn-requestid": "04191d3b-0efc-4a75-95d4-99616281d3f8",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [70]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = ranking_solution_arn
)
ranking_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-60498/f669db6d",
  "ResponseMetadata": {
    "RequestId": "d6e88695-aba5-41dd-a9bc-123ded7eb2e6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 07:18:27 GMT",
      "x-amzn-requestid": "d6e88695-aba5-41dd-a9bc-123ded7eb2e6",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 모델 생성 확인 하기 
아래 코드 실행하여 모델 생성 여부를 확인합니다. 

In [71]:
%%time


max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    ##user_personalization Status
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = user_personalization_solution_version_arn
    )  
    status_user_per = describe_solution_version_response["solutionVersion"]["status"]
    print("user_personalization SolutionVersion: {}".format(status_user_per))
    
    ##Popularity Status
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = popularity_solution_version_arn
    )  
    status_pop = describe_solution_version_response["solutionVersion"]["status"]
    print("Popularity SolutionVersion: {}".format(status_pop))
    
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_solution_version_arn
    )  
    status_hrnn = describe_solution_version_response["solutionVersion"]["status"]
    print("HRNN SolutionVersion: {}".format(status_hrnn))

    #hrnn_meta status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_meta_solution_version_arn
    )
    status_hrnn_meta = describe_solution_version_response["solutionVersion"]["status"]    
    print("HRNN Meta SolutionVersion: {}".format(status_hrnn_meta))
    
    
    #hrnn_coldstart status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_coldstart_solution_version_arn
    )
    status_hrnn_cs = describe_solution_version_response["solutionVersion"]["status"]    
    print("HRNN ColdStart SolutionVersion: {}".format(status_hrnn_cs))
    
    #sims status
    describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = sims_solution_version_arn
    )
    status_sims = describe_solution_version_response["solutionVersion"]["status"]
    print("Sims SolutionVersion: {}".format(status_sims))
    
    #ranking status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = ranking_solution_version_arn
    )
    status_ranking= describe_solution_version_response["solutionVersion"]["status"]    
    print("Ranking SolutionVersion: {}".format(status_ranking))    
    
    if (status_user_per == "ACTIVE" or status_user_per == "CREATE FAILED") &\
       (status_pop == "ACTIVE" or status_pop == "CREATE FAILED") &\
       (status_hrnn == "ACTIVE" or status_hrnn == "CREATE FAILED") &\
       (status_hrnn_meta == "ACTIVE" or status_hrnn_meta == "CREATE FAILED") &\
       (status_hrnn_cs == "ACTIVE" or status_hrnn_cs == "CREATE FAILED") &\
       (status_sims == "ACTIVE" or status_sims == "CREATE FAILED") &\
       (status_ranking == "ACTIVE" or status_ranking == "CREATE FAILED"):
        break
    print("-------------------------------------->")
    time.sleep(60)

print("All solution creation completed")

user_personalization SolutionVersion: CREATE PENDING
Popularity SolutionVersion: CREATE PENDING
HRNN SolutionVersion: CREATE PENDING
HRNN Meta SolutionVersion: CREATE PENDING
HRNN ColdStart SolutionVersion: CREATE PENDING
Sims SolutionVersion: CREATE PENDING
Ranking SolutionVersion: CREATE PENDING
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN Meta SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN Meta SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking Sol

### Store variables

In [72]:

%store user_personalization_solution_arn
%store user_personalization_solution_version_arn

%store popularity_solution_arn
%store popularity_solution_version_arn

%store hrnn_solution_version_arn
%store hrnn_solution_arn

%store hrnn_meta_solution_version_arn
%store hrnn_meta_solution_arn


%store hrnn_coldstart_solution_version_arn
%store hrnn_coldstart_solution_arn

%store sims_solution_version_arn
%store sims_solution_arn

%store ranking_solution_version_arn
%store ranking_solution_arn


Stored 'user_personalization_solution_arn' (str)
Stored 'user_personalization_solution_version_arn' (str)
Stored 'popularity_solution_arn' (str)
Stored 'popularity_solution_version_arn' (str)
Stored 'hrnn_solution_version_arn' (str)
Stored 'hrnn_solution_arn' (str)
Stored 'hrnn_meta_solution_version_arn' (str)
Stored 'hrnn_meta_solution_arn' (str)
Stored 'hrnn_coldstart_solution_version_arn' (str)
Stored 'hrnn_coldstart_solution_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'sims_solution_arn' (str)
Stored 'ranking_solution_version_arn' (str)
Stored 'ranking_solution_arn' (str)
